In [1]:
import pandas as pd
import yaml
from collections import Counter
from pathlib import Path
from sklearn.model_selection import KFold
from ultralytics import YOLO
import os
import glob
import shutil

In [2]:
TARGET_IMAGES_PATH = './images/'
TARGET_LABELS_PATH = './labels/'

image_paths = glob.glob(TARGET_IMAGES_PATH + "*.jpg")
label_paths = glob.glob(TARGET_LABELS_PATH + "*.txt")

print(image_paths)
print(label_paths)


# source_folder_path = "./axial/cross_data/{split}/{type}"
# splits = ["train", "valid"]
# types = ["images/*.jpg", "labels/*.txt"]

# os.makedirs(TARGET_IMAGES_PATH, exist_ok=True)
# os.makedirs(TARGET_LABELS_PATH, exist_ok=True)

# image_paths = list()
# label_paths = list()

# for split in splits:
#     for data_type in types:
#         files = glob.glob(source_folder_path.format(split=split, type=data_type))
#         for file_ in files:
#             if "image" in data_type:
#                 shutil.copy(file_, TARGET_IMAGES_PATH)
#             else:
#                 shutil.copy(file_, TARGET_LABELS_PATH)

# # Store image and label paths for future use
# image_paths = glob.glob(TARGET_IMAGES_PATH + "*.jpg")
# label_paths = glob.glob(TARGET_LABELS_PATH + "*.txt")

# print(f"Copied {len(image_paths)} images to {TARGET_IMAGES_PATH}")
# print(f"Copied {len(label_paths)} labels to {TARGET_LABELS_PATH}")

['./images/00281_116.jpg', './images/00095_170.jpg', './images/00095_188.jpg', './images/00322_115.jpg', './images/00123_10.jpg', './images/00124_9.jpg', './images/00124_10.jpg', './images/00097_79.jpg', './images/00281_124.jpg', './images/00304_113.jpg', './images/00360_102.jpg', './images/00172_102.jpg', './images/00317_26.jpg', './images/00359_97.jpg', './images/00296_132.jpg', './images/00123_13.jpg', './images/00303_85.jpg', './images/00340_121.jpg', './images/00282_83.jpg', './images/00176_89.jpg', './images/00165_123.jpg', './images/00350_139.jpg', './images/00270_11.jpg', './images/00240_144.jpg', './images/00291_24.jpg', './images/00130_114.jpg', './images/00359_115.jpg', './images/00112_10.jpg', './images/00317_22.jpg', './images/00263_117.jpg', './images/00311_105.jpg', './images/00328_90.jpg', './images/00123_15.jpg', './images/00097_80.jpg', './images/00151_10.jpg', './images/00273_109.jpg', './images/00169_127.jpg', './images/00306_102.jpg', './images/00099_154.jpg', './i

In [3]:
dataset_path = Path("./")  # replace with 'path/to/dataset' for your custom data
labels = sorted(dataset_path.rglob("*labels/*.txt"))  # all data in 'labels'
labels

[PosixPath('labels/00095_170.txt'),
 PosixPath('labels/00095_176.txt'),
 PosixPath('labels/00095_188.txt'),
 PosixPath('labels/00095_200.txt'),
 PosixPath('labels/00095_212.txt'),
 PosixPath('labels/00095_224.txt'),
 PosixPath('labels/00095_226.txt'),
 PosixPath('labels/00095_236.txt'),
 PosixPath('labels/00097_65.txt'),
 PosixPath('labels/00097_68.txt'),
 PosixPath('labels/00097_70.txt'),
 PosixPath('labels/00097_72.txt'),
 PosixPath('labels/00097_74.txt'),
 PosixPath('labels/00097_76.txt'),
 PosixPath('labels/00097_77.txt'),
 PosixPath('labels/00097_79.txt'),
 PosixPath('labels/00097_80.txt'),
 PosixPath('labels/00097_81.txt'),
 PosixPath('labels/00097_83.txt'),
 PosixPath('labels/00099_129.txt'),
 PosixPath('labels/00099_130.txt'),
 PosixPath('labels/00099_137.txt'),
 PosixPath('labels/00099_139.txt'),
 PosixPath('labels/00099_145.txt'),
 PosixPath('labels/00099_149.txt'),
 PosixPath('labels/00099_154.txt'),
 PosixPath('labels/00104_14.txt'),
 PosixPath('labels/00104_15.txt'),
 Posi

In [4]:
yaml_file = './no_val.yaml'  # your data YAML with data directories and names dictionary
with open(yaml_file, 'r', encoding="utf8") as y:
    classes = yaml.safe_load(y)['names']
cls_idx = list(range(len(classes)))
print(list(zip(classes, cls_idx)))

[('negative', 0), ('positive', 1)]


In [5]:
indx = [l.stem for l in labels] # uses base filename as ID (no extension)
labels_df = pd.DataFrame([], columns=cls_idx, index=indx)
labels_df

,0,1
00095_170,NaN,NaN
00095_176,NaN,NaN
00095_188,NaN,NaN
00095_200,NaN,NaN
00095_212,NaN,NaN
...,...,...
00359_97,NaN,NaN
00360_102,NaN,NaN
00360_87,NaN,NaN
00360_90,NaN,NaN


In [6]:
for label in labels:
    lbl_counter = Counter()

    with open(label,'r') as lf:
        lines = lf.readlines()

    for l in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(l.split(' ')[0])] += 1

    labels_df.loc[label.stem] = lbl_counter

labels_df = labels_df.fillna(0.0) # replace `nan` values with `0.0`
labels_df

/tmp/ipykernel_43041/3131205890.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labels_df = labels_df.fillna(0.0) # replace `nan` values with `0.0`


,0,1
00095_170,1.0,0.0
00095_176,1.0,0.0
00095_188,1.0,0.0
00095_200,1.0,0.0
00095_212,1.0,0.0
...,...,...
00359_97,0.0,1.0
00360_102,0.0,1.0
00360_87,0.0,1.0
00360_90,0.0,1.0


In [7]:
ksplit = 5
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)   # setting random_state for repeatable results

kfolds = list(kf.split(labels_df))

In [8]:
folds = [f'split_{n}' for n in range(1, ksplit + 1)]
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1E-7)
    fold_lbl_distrb.loc[f'split_{n}'] = ratio

In [9]:
fold_lbl_distrb

,0,1
split_1,0.28125,0.248322
split_2,0.294737,0.207792
split_3,0.268041,0.223684
split_4,0.194175,0.300699
split_5,0.217822,0.273973


In [10]:
import os
import shutil
import yaml
from pathlib import Path

kfold_base_path = Path('./kfold_axial')
shutil.rmtree(kfold_base_path) if kfold_base_path.is_dir() else None  # 기존 폴더가 있으면 삭제
os.makedirs(str(kfold_base_path))  # 새 폴더 생성

yaml_paths = list()
train_txt_paths = list()
val_txt_paths = list()

# 이미지 및 라벨 파일 경로를 절대 경로로 변경
absolute_image_paths = [str(Path(p).absolute()) for p in image_paths]

for i, (train_idx, val_idx) in enumerate(kfolds):
    # Get image paths for train-val split
    train_paths = [absolute_image_paths[j] for j in train_idx]
    val_paths = [absolute_image_paths[j] for j in val_idx]
    
    # Create text files to store image paths
    train_txt = kfold_base_path / f"train_{i}.txt"
    val_txt = kfold_base_path / f"val_{i}.txt"
    
    # Write images paths for training and validation in split i
    with open(str(train_txt), 'w') as f:
        f.writelines(s + '\n' for s in train_paths)
    with open(str(val_txt), 'w') as f:
        f.writelines(s + '\n' for s in val_paths)
    
    train_txt_paths.append(str(train_txt))
    val_txt_paths.append(str(val_txt))
    
    # Create dataset yaml file
    dataset_yaml = kfold_base_path / f"data_{i}.yaml"
    yaml_content = {
        'path': str(kfold_base_path),  # Root path for dataset
        'train': str(train_txt),       # Train txt file path
        'val': str(val_txt),           # Validation txt file path
        'test': "/home/under1/Detect/jeongui/cross_val/axial/yolo_data/images/test",
        'names': classes               # Class names
    }
    with open(dataset_yaml, 'w') as f:
        yaml.dump(yaml_content, f)
    
    yaml_paths.append(str(dataset_yaml))

# 디버깅을 위한 출력 추가
for i in range(len(kfolds)):
    print(f"Using dataset YAML: {yaml_paths[i]}")
    with open(yaml_paths[i], 'r') as f:
        print(f.read())
    print(f"Train file contents: {train_txt_paths[i]}")
    with open(train_txt_paths[i], 'r') as f:
        print(f.read())
    print(f"Val file contents: {val_txt_paths[i]}")
    with open(val_txt_paths[i], 'r') as f:
        print(f.read())


Using dataset YAML: kfold_axial/data_0.yaml
names:
- negative
- positive
path: kfold_axial
test: /home/under1/Detect/jeongui/cross_val/axial/yolo_data/images/test
train: kfold_axial/train_0.txt
val: kfold_axial/val_0.txt

Train file contents: kfold_axial/train_0.txt
/home/under1/Detect/jeongui/cross_val/axial/cross_data/images/00095_170.jpg
/home/under1/Detect/jeongui/cross_val/axial/cross_data/images/00095_188.jpg
/home/under1/Detect/jeongui/cross_val/axial/cross_data/images/00322_115.jpg
/home/under1/Detect/jeongui/cross_val/axial/cross_data/images/00123_10.jpg
/home/under1/Detect/jeongui/cross_val/axial/cross_data/images/00124_9.jpg
/home/under1/Detect/jeongui/cross_val/axial/cross_data/images/00124_10.jpg
/home/under1/Detect/jeongui/cross_val/axial/cross_data/images/00097_79.jpg
/home/under1/Detect/jeongui/cross_val/axial/cross_data/images/00281_124.jpg
/home/under1/Detect/jeongui/cross_val/axial/cross_data/images/00304_113.jpg
/home/under1/Detect/jeongui/cross_val/axial/cross_data

In [11]:
from IPython.display import clear_output
import time

batch = 16
project = 'kfold_demo'
epochs = 200

results = list()

for i in range(ksplit):
    model = YOLO('yolov8n.pt')
    dataset_yaml =yaml_paths[i]
    print(f"Training for fold={i} using {dataset_yaml}")
    model.train(data=dataset_yaml, batch=batch, project=project, epochs=epochs, verbose=False, workers=28)
    result = model.metrics # Metrics on validation set
    results.append(result) # save output metrics for further analysis
    clear_output()

In [12]:
metric_values = dict()

for result in results:
    for metric, metric_val in result.results_dict.items():
        if metric not in metric_values:
            metric_values[metric] = []
        metric_values[metric].append(metric_val)

metric_df = pd.DataFrame.from_dict(metric_values)
visualize_metric = ['mean', 'std', 'min', 'max']
metric_df.describe().loc[visualize_metric]

,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),fitness
mean,0.922767,0.907826,0.949397,0.733130,0.754757
std,0.054322,0.017783,0.023106,0.021516,0.020725
min,0.846578,0.885823,0.913975,0.703299,0.724366
max,0.964287,0.925184,0.972140,0.759771,0.779781


In [13]:
metric_values

{'metrics/precision(B)': [0.9642874995442743,
  0.959333242374178,
  0.8465779556130342,
  0.9601331909898857,
  0.8835049973497935],
 'metrics/recall(B)': [0.9219621301974243,
  0.8858225108225108,
  0.8922491849215988,
  0.9139145986046013,
  0.9251840462117676],
 'metrics/mAP50(B)': [0.9721395643671475,
  0.9617087471590213,
  0.9139746365959278,
  0.9392894782245469,
  0.9598735602005161],
 'metrics/mAP50-95(B)': [0.7332884399031335,
  0.7235221565616514,
  0.7032988344903615,
  0.745769984837991,
  0.7597710050985907],
 'fitness': [0.7571735523495349,
  0.7473408156213884,
  0.7243664147009181,
  0.7651219341766466,
  0.7797812606087833]}